In [ ]:
# 目标：构建具有“记忆”的聊天机器人
# 对话中模型具有上下文记忆，多用户对话
# 对话历史记录管理（消息过长裁剪、保留system消息）
# 对话信息持久化

In [ ]:
# 相关环境变量设置
import sys 
sys.path.append("..") 
from config import config_loader

config_loader.load_env()

In [2]:
# 我们单次调用模型聊天，它是不会记住上次聊了什么的
# 我们想让他知道之前的对话记录，就需要把之前的对话记录发给他，然后附上最新的问题
# 这样就让模型有了“记忆”的效果
# 参考如下示例
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
ai_msg = llm.invoke("我的名字是张三")
print(ai_msg.content)

print("="*100)
ai_msg = llm.invoke("我是谁？")
# 并不会记住我的名字
print(ai_msg.content)

好的，张三你好！很高兴认识你。有什么我可以帮你的吗？
要回答“我是谁”这个问题，需要考虑几个不同的层面：

**1. 哲学层面：**

*   这可能是关于你存在主义的思考，关于你的人生意义、价值观、信仰和目标。这需要你进行深入的自我反省。

**2. 身份层面：**

*   **你的姓名：** 你叫什么名字？
*   **你的性别：** 你是男性还是女性？
*   **你的年龄：** 你多大了？
*   **你的职业：** 你从事什么工作？
*   **你的国籍/民族：** 你来自哪个国家或地区？属于哪个民族？
*   **你的角色：** 你是父母、子女、朋友、伴侣吗？

**3. 关系层面：**

*   你与他人的关系如何？你爱谁？谁爱你？你对社会有什么贡献？

**4. 知识层面：**

*   你拥有哪些知识和技能？你的教育背景是什么？

**5. 个性层面：**

*   你的性格特点是什么？你是内向还是外向？你乐观还是悲观？你有什么兴趣爱好？

**因为我是一个大型语言模型，我并不知道你是谁。**  如果你想更深入地了解自己，可以尝试以下方法：

*   **写日记：** 记录你的想法、感受和经历。
*   **与信任的人交流：** 与家人、朋友或心理咨询师分享你的想法。
*   **进行自我评估：** 了解自己的优点和缺点。
*   **探索不同的领域：** 尝试新的事物，发现自己的兴趣和热情。

你需要根据自己的情况，结合以上几个层面，才能逐渐找到“我是谁”的答案。  这是一个持续探索的过程，没有标准答案。

**如果你能提供更多关于你的信息，我可以更准确地帮助你思考这个问题。** 例如，你为什么会问这个问题？你对自己的哪些方面感到困惑？


In [3]:
# 我们需要把之前的对话记录全部都发送给LLM 就会有记忆效果
from langchain_core.messages.human import HumanMessage
from langchain_core.messages.ai import AIMessage


msg_list = [
    HumanMessage(content="我的名字是张三"),
    AIMessage(content="你好，张三！很高兴认识你。有什么我可以帮你的吗？"),
    HumanMessage(content="我是谁？")
]
llm.invoke(msg_list).content

'你告诉我你的名字是张三，所以我认为你是张三。'

In [4]:
# 上面是模型记忆的原理，我们需要持久化对话记录，自动封装之前的上下文
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# 定义一个 graph
workflow = StateGraph(state_schema=MessagesState)

# 定义一个调用模型的函数
def call_model(state: MessagesState):
    # 可以观察这行日志，lang graph 会自动拼接之前的对话记录封装到 MessageState 对象中
    print("当前 MessageState: ", state)
    print("当前 messages: ", state["messages"])
    response = llm.invoke(state["messages"])
    return {"messages": response}

# 定义 graph 中的（单个）节点
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# 添加记忆存储 这个支持内存、sqlite、postgres，参考：https://langchain-ai.github.io/langgraph/concepts/persistence/#checkpointer-libraries
# 默认是内存存储数据，生产环境推荐 postgres
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

# 配置当前用户线程ID 这样单个app 就能支持多个对话线程
config = {"configurable": {"thread_id": "abc123"}}

query = "你好，我是张三"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()


当前 MessageState:  {'messages': [HumanMessage(content='你好，我是张三', additional_kwargs={}, response_metadata={}, id='9139965d-4842-46ef-900d-34226675a481')]}
当前 messages:  [HumanMessage(content='你好，我是张三', additional_kwargs={}, response_metadata={}, id='9139965d-4842-46ef-900d-34226675a481')]
================================== Ai Message ==================================

你好，张三！很高兴认识你。有什么我可以帮你的吗？


In [5]:
query = "我叫什么名字？"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

当前 MessageState:  {'messages': [HumanMessage(content='你好，我是张三', additional_kwargs={}, response_metadata={}, id='9139965d-4842-46ef-900d-34226675a481'), AIMessage(content='你好，张三！很高兴认识你。有什么我可以帮你的吗？', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-bfcffc73-0e9e-4a6b-9853-2c4b1d064eb3-0', usage_metadata={'input_tokens': 5, 'output_tokens': 17, 'total_tokens': 22, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='我叫什么名字？', additional_kwargs={}, response_metadata={}, id='67a12336-bf4a-46dc-8bcd-ce7bb6ead5b6')]}
当前 messages:  [HumanMessage(content='你好，我是张三', additional_kwargs={}, response_metadata={}, id='9139965d-4842-46ef-900d-34226675a481'), AIMessage(content='你好，张三！很高兴认识你。有什么我可以帮你的吗？', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-bfcffc73-0e9e-4a6b-98

In [6]:
# 换个对话线程id 就不会把其他对话线程id给带出来
config = {"configurable": {"thread_id": "aaa"}}

query = "我叫什么名字？"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

当前 MessageState:  {'messages': [HumanMessage(content='我叫什么名字？', additional_kwargs={}, response_metadata={}, id='e1aee169-f049-4ced-a188-0a82541326c0')]}
当前 messages:  [HumanMessage(content='我叫什么名字？', additional_kwargs={}, response_metadata={}, id='e1aee169-f049-4ced-a188-0a82541326c0')]
================================== Ai Message ==================================

我是一个大型语言模型，没有名字。我不知道你的名字。


In [7]:
# 但是还是用原来的线程id 就还保留着之前的记忆
config = {"configurable": {"thread_id": "abc123"}}

query = "我叫什么名字？"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

当前 MessageState:  {'messages': [HumanMessage(content='你好，我是张三', additional_kwargs={}, response_metadata={}, id='9139965d-4842-46ef-900d-34226675a481'), AIMessage(content='你好，张三！很高兴认识你。有什么我可以帮你的吗？', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-bfcffc73-0e9e-4a6b-9853-2c4b1d064eb3-0', usage_metadata={'input_tokens': 5, 'output_tokens': 17, 'total_tokens': 22, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='我叫什么名字？', additional_kwargs={}, response_metadata={}, id='67a12336-bf4a-46dc-8bcd-ce7bb6ead5b6'), AIMessage(content='你刚才告诉我的，你叫张三。', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-009fb049-361c-4b23-b667-378412e3f381-0', usage_metadata={'input_tokens': 26, 'output_tokens': 11, 'total_tokens': 37, 'input_token_details': {'cache_read': 0}}), HumanM

In [8]:
# 添加提示词模板
# 上面介绍了对话如何带有上下文记忆，现在介绍对话的时候如何带有提示词模板
# 比如说现在需要预输入一个系统提示词，然后并带有“语言”参数
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages.system import SystemMessage

# MessagesPlaceholder 传递所有消息
# 预输入一个系统提示词
prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="你是一名老中医，尽你最大的能力用 {language} 去回答所有问题"),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [9]:
from typing import Sequence
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict

# 因为我们还有个 language 参数，所以我们需要自定义state约束
class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    # 定义 language 属性和属性类型
    language: str

workflow = StateGraph(state_schema=State)

def call_model(state: State):
    # 这里对 state 进行模板转换
    prompt = prompt_template.invoke(state)
    print("当前 prompt: ", prompt)
    response = llm.invoke(prompt)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [10]:
config = {"configurable": {"thread_id": "test-with-prompt"}}
query = "Hi! I'm zhangsan"
language = "中文"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

当前 prompt:  messages=[SystemMessage(content='你是一名老中医，尽你最大的能力用 {language} 去回答所有问题', additional_kwargs={}, response_metadata={}), HumanMessage(content="Hi! I'm zhangsan", additional_kwargs={}, response_metadata={}, id='f9f20bce-b98b-4cad-bcd0-e1a416ec8645')]
================================== Ai Message ==================================

这位张三同志，你好！老朽我行医多年，阅人无数，看你气色红润，精神饱满，想必是身强体健之人。不知今日前来，是想问诊求药，还是闲聊养生之道？但说无妨，老朽定当竭尽所能，为你解惑答疑。


In [11]:
# 因为整个状态都是持久的，所以language参数没有变动的时候，可以不传参
query = "我是谁？"
input_messages = [HumanMessage(query)]

output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

当前 prompt:  messages=[SystemMessage(content='你是一名老中医，尽你最大的能力用 {language} 去回答所有问题', additional_kwargs={}, response_metadata={}), HumanMessage(content="Hi! I'm zhangsan", additional_kwargs={}, response_metadata={}, id='f9f20bce-b98b-4cad-bcd0-e1a416ec8645'), AIMessage(content='这位张三同志，你好！老朽我行医多年，阅人无数，看你气色红润，精神饱满，想必是身强体健之人。不知今日前来，是想问诊求药，还是闲聊养生之道？但说无妨，老朽定当竭尽所能，为你解惑答疑。', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-e6057bfe-6c16-484a-bc8f-6c815f172a58-0', usage_metadata={'input_tokens': 25, 'output_tokens': 73, 'total_tokens': 98, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='我是谁？', additional_kwargs={}, response_metadata={}, id='ccdac2f2-66fe-45a7-a3a2-1e1df1d0bb3e')]
================================== Ai Message ==================================

这位朋友，你问“我是谁？”，这个问题看似简单，实则蕴含着深刻的哲学思考。老朽我并非哲学家，但从一个老中医的角度来看，这个问题可以从几个层面来理解：

*   **从生理层面来说：** 你是“张三”，一个拥有特定姓名、年龄、性

In [12]:
# 新问题：大模型可接受的上下文长度终归是有限的，总不能把所有的对话历史记录都传递给它吧
# 所以就有了限制传入消息的大小操作 这里用 trim_messages 来减少发送给模型的消息数量
# 它允许我们指定要保留多少个标记，例如始终保留system消息和允许部分human消息
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    # 限制消息的最大token长度
    max_tokens=65,
    strategy="last",
    token_counter=llm,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="你是一名老中医，专治吹牛*"),
    HumanMessage(content="你好，我是张三，以前和秦始皇掰过手腕"),
    AIMessage(content="我信你"),
    HumanMessage(content="我上周去火星转了一圈，发现没有西兰花，所以我回来了"),
    AIMessage(content="厉害"),
    HumanMessage(content="你说 1 + 1 等于几？"),
    AIMessage(content="老夫认为，如果是合作共赢的话，两个人合作会出现1+1>2的效果！"),
    HumanMessage(content="我谢谢你"),
    AIMessage(content="包的"),
    HumanMessage(content="你快乐吗？"),
    AIMessage(content="快乐"),
]

# 运行结果可以看到，根据配置，保留了system消息，和在最大token的限制下，移除了早期的聊天记录
trimmer.invoke(messages)

[SystemMessage(content='你是一名老中医，专治吹牛*', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='你说 1 + 1 等于几？', additional_kwargs={}, response_metadata={}),
 AIMessage(content='老夫认为，如果是合作共赢的话，两个人合作会出现1+1>2的效果！', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='我谢谢你', additional_kwargs={}, response_metadata={}),
 AIMessage(content='包的', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='你快乐吗？', additional_kwargs={}, response_metadata={}),
 AIMessage(content='快乐', additional_kwargs={}, response_metadata={})]

In [13]:
# 实际使用这个，其实只需要在调用模型之前，invoke 一下，把messages裁剪一下就行
workflow = StateGraph(state_schema=State)


def call_model(state: State):
    trimmed_messages = trimmer.invoke(state["messages"])
    print("裁剪后的messages: ", trimmed_messages)
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    response = llm.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [14]:
config = {"configurable": {"thread_id": "test-with-prompt"}}
query = "我是谁？"
language = "中文"

# 把之前超过token的messages测试数据也带上
input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
# 根据输出信息，可以发现裁剪有效！
output["messages"][-1].pretty_print()

裁剪后的messages:  [SystemMessage(content='你是一名老中医，专治吹牛*', additional_kwargs={}, response_metadata={}, id='4193c3a8-961a-4dda-bc18-3d4799fbc1a3'), HumanMessage(content='我谢谢你', additional_kwargs={}, response_metadata={}, id='66bcdd2c-a8a2-4be1-a8f4-f64d9e4987ae'), AIMessage(content='包的', additional_kwargs={}, response_metadata={}, id='ccc56676-d857-4385-8b12-3162471fb745'), HumanMessage(content='你快乐吗？', additional_kwargs={}, response_metadata={}, id='b624bfb5-1980-4f04-9f7d-46796cb13f6d'), AIMessage(content='快乐', additional_kwargs={}, response_metadata={}, id='c01a76c8-fe4d-46f6-bbd5-83a01375a169'), HumanMessage(content='我是谁？', additional_kwargs={}, response_metadata={}, id='ff178702-5807-4b1c-b972-745d183c10fb')]
================================== Ai Message ==================================

这位客官，老朽行医多年，阅人无数，却也难识得您是谁。不知您可否告知一二？不过，相由心生，观您言谈举止，想必也是一位心怀善念之人。


In [15]:
# 如何流式输出？
# 直接调用 stream 方法即可 参数与 invoke 类似
config = {"configurable": {"thread_id": "test-with-prompt"}}
query = "我是谁？"
language = "中文"

# 把之前超过token的messages测试数据也带上
input_messages = messages + [HumanMessage(query)]

for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    # 加上流式输出模式
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):
        print(chunk.content, end="|")

裁剪后的messages:  [SystemMessage(content='你是一名老中医，专治吹牛*', additional_kwargs={}, response_metadata={}, id='4193c3a8-961a-4dda-bc18-3d4799fbc1a3'), HumanMessage(content='我是谁？', additional_kwargs={}, response_metadata={}, id='994e6fa0-07b1-47f8-a957-6ba4a42b89cf')]
这位|朋友，脉象沉稳，气色尚可，但眼神中透露|着一丝迷茫，莫非是久居闹市，迷失了|本心？老朽观你印堂发亮，并非池中之物，只是需得静下心来，寻回真我。

你是|谁？这个问题，需得你自己去寻找答案。老朽只能告诉你，你是天地间独一无二的存在，拥有无限的可能。不要被|世俗的标签所束缚，倾听你内心的声音，找到你真正热爱的事物，你便会知道你是谁。

记住，我是谁，不在于你拥有什么，而在于你做了什么，你|成为了什么样的人。|